In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-s27x10r7
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-s27x10r7
  Resolved https://github.com/facebookresearch/detectron2.git to commit fd27788985af0f4ca800bca563acdb700bb890e2
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 33.9 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp312-cp312-linux_x86_64.whl size=7082985 sha256=ae315e8830bd223f852b49084a43c4bba210e9d6ff4f4acf41e2134a32fab4a7
  Stored in directory: /tmp/pip-ephem-wheel-cache-8cagmw8l/wheels/d3/6e/bd/1969578f1456a6be

In [70]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.data.datasets import register_coco_instances
from detectron2 import model_zoo
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
import os, json

In [71]:
BASE_PATH = "/content/drive/MyDrive/proyecto final maia"

In [72]:
from PIL import Image

def add_images_size(images):
  for img in images:
    img_path = IMG_DIR / img["file_name"]

    with Image.open(img_path) as pil:
      w, h = pil.size

    img["width"] = w
    img["height"] = h

  return images

In [73]:
import json, random, re

from pathlib import Path
from collections import defaultdict, Counter

ANN_IN  = Path(f"{BASE_PATH}/annotations_filtered.json")
IMG_DIR = Path(f"{BASE_PATH}/images")  # not used here but handy to keep
OUT_DIR = Path(f"{BASE_PATH}/splits/final")

coco = json.loads(ANN_IN.read_text(encoding="utf-8"))

images = coco["images"]
anns = coco["annotations"]
cats = coco["categories"]

cat_id_to_name = {c["id"]: c["name"] for c in cats}

images = add_images_size(images)

def write_coco(name, img_id_set):
    imgs = [im for im in images if im["id"] in img_id_set]
    keep_ids = {im["id"] for im in imgs}
    anns_keep = [a for a in anns if a["image_id"] in keep_ids]
    out = {"images": imgs, "annotations": anns_keep, "categories": cats}
    out_path = OUT_DIR / f"{name}.json"
    out_path.write_text(json.dumps(out, indent=2), encoding="utf-8")
    return out_path, len(imgs), len(anns_keep)

# split validation
final_splits = f"{BASE_PATH}/data/images/"

for split in ["train", "val"]:
    image_id_set = [int(re.search(r"\d+", fname).group()) for fname in os.listdir(f"{final_splits}/{split}/")]
    p, ni, na = write_coco(split, image_id_set)
    print(f"{split}: images={ni}, anns={na}, -> {p}")


for split in ["train", "val"]:
    with open(OUT_DIR / f"{split}.json") as f:
        d = json.load(f)
    cnt = Counter(cat_id_to_name[a["category_id"]] for a in d["annotations"])
    print(split, "label counts (top 10):", cnt.most_common(10))

train: images=114, anns=1760, -> /content/drive/MyDrive/proyecto final maia/splits/final/train.json
val: images=26, anns=408, -> /content/drive/MyDrive/proyecto final maia/splits/final/val.json
train label counts (top 10): [('T1', 108), ('T2', 107), ('T3', 107), ('T4', 107), ('T5', 106), ('T6', 106), ('T7', 106), ('T8', 106), ('T9', 106), ('T10', 105)]
val label counts (top 10): [('T1', 25), ('T2', 25), ('T3', 25), ('T4', 25), ('T5', 24), ('T6', 24), ('T7', 24), ('T8', 24), ('T9', 24), ('T10', 24)]


In [74]:
ann_train = str(OUT_DIR / "train.json")
ann_val   = str(OUT_DIR / "val.json")

In [75]:
import json
from pathlib import Path
from datetime import datetime

def patch_coco(path_in, path_out=None):
    path_in = Path(path_in)
    if path_out is None: path_out = path_in  # in-place
    coco = json.loads(path_in.read_text(encoding="utf-8"))

    # Required top-level keys for some COCO evaluators
    coco.setdefault("info", {
        "description": "Spine vertebrae dataset (autogenerated)",
        "version": "1.0",
        "year": datetime.now().year,
        "date_created": datetime.now().strftime("%Y-%m-%d"),
        "contributor": ""
    })
    coco.setdefault("licenses", [])  # many tools expect the key to exist

    # Make sure categories have at least name/id/supercategory
    for c in coco.get("categories", []):
        c.setdefault("supercategory", "spine")

    Path(path_out).write_text(json.dumps(coco, indent=2), encoding="utf-8")
    print(f"Patched: {path_out}")

patch_coco(ann_train)
patch_coco(ann_val)

Patched: /content/drive/MyDrive/proyecto final maia/splits/final/train.json
Patched: /content/drive/MyDrive/proyecto final maia/splits/final/val.json


In [76]:
# Register datasets
register_coco_instances("spine_train", {}, ann_train, IMG_DIR)
register_coco_instances("spine_val",   {}, ann_val,   IMG_DIR)

with open(ann_train) as f:
    NUM_CLASSES = len(json.load(f)["categories"])
print("NUM_CLASSES =", NUM_CLASSES)  # should be 17 (T1..T12 + L1..L5)

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(
    "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
))
cfg.DATASETS.TRAIN = ("spine_train",)
cfg.DATASETS.TEST  = ("spine_val",)
cfg.DATALOADER.NUM_WORKERS = 8

# Input sizes (keep detail; dataset is small)
cfg.INPUT.MIN_SIZE_TRAIN = (800, 1024)
cfg.INPUT.MAX_SIZE_TRAIN = 1536
cfg.INPUT.MIN_SIZE_TEST  = 1024
cfg.INPUT.MAX_SIZE_TEST  = 1536
cfg.INPUT.RANDOM_FLIP = "horizontal"

# Solver
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 5.25e-4
cfg.SOLVER.WARMUP_ITERS = 750
cfg.SOLVER.MAX_ITER = 5000      # ~small dataset; adjust after seeing curves
cfg.SOLVER.STEPS = []           # keep LR constant initially
cfg.SOLVER.AMP.ENABLED = True   # mixed precision
cfg.SOLVER.CLIP_GRADIENTS.ENABLED = True

# Model
cfg.MODEL.ROI_HEADS.NUM_CLASSES = NUM_CLASSES
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(
    "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
)

cfg.OUTPUT_DIR = "./out_maskrcnn"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

NUM_CLASSES = 17


In [77]:
class Trainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name):
        return COCOEvaluator(dataset_name, output_dir=cfg.OUTPUT_DIR)

In [78]:
trainer = Trainer(cfg)
trainer.resume_or_load(False)

[11/23 13:44:57 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

/usr/local/lib/python3.12/dist-packages/detectron2/engine/train_loop.py:471: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  grad_scaler = GradScaler()
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


In [79]:
trainer.train()

[11/23 13:45:01 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.12/dist-packages/detectron2/engine/train_loop.py:490: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=self.precision):
/usr/local/lib/python3.12/dist-packages/torch/optim/lr_scheduler.py:192: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/detectron2/engine/train_loop.py:490: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=self.precision):
/usr/local/lib/python3.12/dist-packages/de

[11/23 13:45:09 d2.utils.events]:  eta: 0:30:24  iter: 19  total_loss: 5.751  loss_cls: 3.056  loss_box_reg: 0.4496  loss_mask: 0.6924  loss_rpn_cls: 1.473  loss_rpn_loc: 0.1292    time: 0.3642  last_time: 0.3517  data_time: 0.0354  last_data_time: 0.0089   lr: 1.3812e-05  max_mem: 4346M


/usr/local/lib/python3.12/dist-packages/detectron2/engine/train_loop.py:490: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=self.precision):
/usr/local/lib/python3.12/dist-packages/detectron2/engine/train_loop.py:490: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=self.precision):
/usr/local/lib/python3.12/dist-packages/detectron2/engine/train_loop.py:490: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=self.precision):
/usr/local/lib/python3.12/dist-packages/detectron2/engine/train_loop.py:490: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=self.precision):
/usr/local/lib/python3.12/dist-packages/dete

[11/23 13:45:17 d2.utils.events]:  eta: 0:32:31  iter: 39  total_loss: 4.627  loss_cls: 2.608  loss_box_reg: 0.713  loss_mask: 0.6922  loss_rpn_cls: 0.4312  loss_rpn_loc: 0.1088    time: 0.3888  last_time: 0.3974  data_time: 0.0101  last_data_time: 0.0102   lr: 2.7798e-05  max_mem: 4504M


...

[11/23 14:17:59 d2.utils.events]:  eta: 0:00:15  iter: 4959  total_loss: 0.2344  loss_cls: 0.03747  loss_box_reg: 0.08316  loss_mask: 0.1025  loss_rpn_cls: 9.108e-05  loss_rpn_loc: 0.01508    time: 0.3983  last_time: 0.3817  data_time: 0.0094  last_data_time: 0.0027   lr: 0.000525  max_mem: 4516M


/usr/local/lib/python3.12/dist-packages/detectron2/engine/train_loop.py:490: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=self.precision):
/usr/local/lib/python3.12/dist-packages/detectron2/engine/train_loop.py:490: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=self.precision):
/usr/local/lib/python3.12/dist-packages/detectron2/engine/train_loop.py:490: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=self.precision):
/usr/local/lib/python3.12/dist-packages/detectron2/engine/train_loop.py:490: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=self.precision):
/usr/local/lib/python3.12/dist-packages/dete

[11/23 14:18:07 d2.utils.events]:  eta: 0:00:07  iter: 4979  total_loss: 0.2492  loss_cls: 0.04118  loss_box_reg: 0.08449  loss_mask: 0.1045  loss_rpn_cls: 0.0001083  loss_rpn_loc: 0.01587    time: 0.3983  last_time: 0.4086  data_time: 0.0097  last_data_time: 0.0061   lr: 0.000525  max_mem: 4516M


/usr/local/lib/python3.12/dist-packages/detectron2/engine/train_loop.py:490: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=self.precision):
/usr/local/lib/python3.12/dist-packages/detectron2/engine/train_loop.py:490: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=self.precision):
/usr/local/lib/python3.12/dist-packages/detectron2/engine/train_loop.py:490: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=self.precision):
/usr/local/lib/python3.12/dist-packages/detectron2/engine/train_loop.py:490: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=self.precision):
/usr/local/lib/python3.12/dist-packages/dete

[11/23 14:18:17 d2.utils.events]:  eta: 0:00:00  iter: 4999  total_loss: 0.2398  loss_cls: 0.03987  loss_box_reg: 0.08138  loss_mask: 0.1058  loss_rpn_cls: 0.0001136  loss_rpn_loc: 0.0153    time: 0.3983  last_time: 0.4225  data_time: 0.0109  last_data_time: 0.0230   lr: 0.000525  max_mem: 4516M
[11/23 14:18:19 d2.engine.hooks]: Overall training speed: 4998 iterations in 0:33:10 (0.3983 s / it)
[11/23 14:18:19 d2.engine.hooks]: Total training time: 0:33:16 (0:00:06 on hooks)
[11/23 14:18:19 d2.data.datasets.coco]: Loaded 26 images in COCO format from /content/drive/MyDrive/proyecto final maia/splits/final/val.json
[11/23 14:18:19 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(1024, 1024), max_size=1536, sample_style='choice')]
[11/23 14:18:19 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[11/23 14:18:19 d2.data.common]: Serializing 26 elements to byte tensors and co

In [80]:
# cell: compute COCO mAP (full) + mAP@50 + IoU + Dice, save CSV
import numpy as np
import cv2
import json
import os
import pandas as pd
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from pycocotools import mask as maskutils
from tqdm import tqdm
from collections import defaultdict
from detectron2.engine import DefaultPredictor

# --- CONFIG: adjust if needed ---
VAL_JSON = f"{BASE_PATH}/splits/final/val.json"
IMG_DIR = f"{BASE_PATH}/images"
WEIGHTS = "./out_maskrcnn/model_final.pth"
CSV_OUT = "./out_maskrcnn/metrics_with_map50.csv"

# --- load GT ---
coco_gt = COCO(VAL_JSON)
cat_list = coco_gt.loadCats(coco_gt.getCatIds())
cat_id_to_name = {c["id"]: c["name"] for c in cat_list}
cat_ids = [c["id"] for c in cat_list]  # order of categories in GT

# --- predictor (assumes cfg is in session) ---
cfg.MODEL.WEIGHTS = WEIGHTS
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
predictor = DefaultPredictor(cfg)

# --- helper metrics ---
def iou_score(pred_mask, gt_mask):
    inter = np.logical_and(pred_mask, gt_mask).sum()
    union = np.logical_or(pred_mask, gt_mask).sum()
    return float(inter) / (float(union) + 1e-6)

def dice_score(pred_mask, gt_mask):
    inter = np.logical_and(pred_mask, gt_mask).sum()
    return (2.0 * float(inter)) / (pred_mask.sum() + gt_mask.sum() + 1e-6)

# accumulators
iou_scores = defaultdict(list)
dice_scores = defaultdict(list)
predictions_coco = []

# --- iterate validation images, collect preds & compute IoU/Dice per GT ---
for img_info in tqdm(coco_gt.dataset["images"], desc="Evaluating val images"):
    img_path = os.path.join(IMG_DIR, img_info["file_name"])
    img = cv2.imread(img_path)
    if img is None:
        print("Warning: missing image:", img_path)
        continue

    outputs = predictor(img)
    instances = outputs["instances"].to("cpu")

    # build predictions list for COCOeval (masks -> RLE)
    if len(instances) > 0:
        masks_np = instances.pred_masks.numpy()            # (N,H,W) bool
        boxes_np = instances.pred_boxes.tensor.numpy()    # (N,4) x1,y1,x2,y2
        classes_np = instances.pred_classes.numpy()       # 0-based
        scores_np = instances.scores.numpy()
    else:
        masks_np = np.zeros((0, img.shape[0], img.shape[1]), dtype=bool)
        boxes_np = np.zeros((0,4))
        classes_np = np.array([], dtype=int)
        scores_np = np.array([], dtype=float)

    for i in range(len(masks_np)):
        mask = masks_np[i].astype(np.uint8)
        # encode to RLE expected by COCO
        rle = maskutils.encode(np.asfortranarray(mask))
        # counts may be bytes; convert to utf-8 string to be JSON serializable
        if isinstance(rle.get("counts"), bytes):
            rle["counts"] = rle["counts"].decode("ascii")
        x1, y1, x2, y2 = boxes_np[i].tolist()
        bbox = [float(x1), float(y1), float(x2 - x1), float(y2 - y1)]
        # map detectron2 0-based class -> COCO category id (here we assume +1 mapping)
        cid = int(classes_np[i]) + 1
        predictions_coco.append({
            "image_id": img_info["id"],
            "category_id": cid,
            "score": float(scores_np[i]),
            "segmentation": rle,
            "bbox": bbox
        })

    # prepare preds grouped by class to speed mask matching
    preds_by_cid = defaultdict(list)
    for p_mask, p_cls in zip(masks_np, classes_np):
        preds_by_cid[int(p_cls) + 1].append(p_mask.astype(bool))

    # compute IoU/Dice per GT annotation
    ann_ids = coco_gt.getAnnIds(imgIds=[img_info["id"]])
    anns = coco_gt.loadAnns(ann_ids)
    for ann in anns:
        gt_mask = coco_gt.annToMask(ann).astype(bool)
        gt_cid = ann["category_id"]
        gt_name = cat_id_to_name.get(gt_cid, str(gt_cid))

        best_iou = 0.0
        best_dice = 0.0
        for p_mask in preds_by_cid.get(gt_cid, []):
            iou = iou_score(p_mask, gt_mask)
            if iou > best_iou:
                best_iou = iou
                best_dice = dice_score(p_mask, gt_mask)
        iou_scores[gt_name].append(best_iou)
        dice_scores[gt_name].append(best_dice)

# --- write predictions json for COCOeval ---
pred_file = "predictions_coco.json"
with open(pred_file, "w") as f:
    json.dump(predictions_coco, f)

# --- COCOeval for full mAP (IoU=0.50:0.95) ---
print("\n--- COCOeval: full mAP (IoU=0.50:0.95) ---")
coco_dt = coco_gt.loadRes(pred_file)
coco_eval = COCOeval(coco_gt, coco_dt, iouType='segm')
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()
stats_full = coco_eval.stats.copy() if coco_eval.stats is not None else [float('nan')]*6

# helper to extract per-category AP from coco_eval
def get_per_category_ap(coco_eval_obj):
    precision = coco_eval_obj.eval.get('precision')  # shape: [TxRxKxAxM] or None
    per_cat = {}
    if precision is None:
        # no precision data
        for cid in cat_ids:
            per_cat[cid] = float('nan')
        return per_cat

    # we'll average across IoU thresholds & recall for area=all and maxDets=100
    area_idx = 0  # area=all
    maxdet_idx = precision.shape[-1] - 1
    for k, cid in enumerate(cat_ids):
        pr = precision[:, :, k, area_idx, maxdet_idx]  # shape (T, R)
        valid = pr[pr > -1]
        per_cat[cid] = float(np.mean(valid)) if valid.size > 0 else float('nan')
    return per_cat

per_cat_ap_full = get_per_category_ap(coco_eval)

# --- Now compute mAP@50 specifically ---
print("\n--- COCOeval: mAP@50 (IoU = 0.50) ---")
# change IoU thresholds to single 0.5 value and re-run
coco_eval.params.iouThrs = np.array([0.5])
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()
stats_50 = coco_eval.stats.copy() if coco_eval.stats is not None else [float('nan')]*6
per_cat_ap_50 = get_per_category_ap(coco_eval)

# --- Build CSV rows per class ---
rows = []
for cid, cname in cat_id_to_name.items():
    iou_list = iou_scores.get(cname, [])
    dice_list = dice_scores.get(cname, [])
    ap_full = per_cat_ap_full.get(cid, float('nan'))
    ap50 = per_cat_ap_50.get(cid, float('nan'))
    row = {
        "class_id": cid,
        "class": cname,
        "AP_coco": float(ap_full),
        "AP50": float(ap50),
        "IoU_mean": float(np.mean(iou_list)) if iou_list else float('nan'),
        "IoU_std": float(np.std(iou_list)) if iou_list else float('nan'),
        "Dice_mean": float(np.mean(dice_list)) if dice_list else float('nan'),
        "Dice_std": float(np.std(dice_list)) if dice_list else float('nan'),
        "n_samples": int(len(iou_list))
    }
    rows.append(row)

# overall
overall_iou = np.mean([s for lst in iou_scores.values() for s in lst]) if any(iou_scores.values()) else float('nan')
overall_dice = np.mean([s for lst in dice_scores.values() for s in lst]) if any(dice_scores.values()) else float('nan')
overall_row = {
    "class_id": 0,
    "class": "Overall",
    "mAP": float(stats_full[0]) if len(stats_full) > 0 else float('nan'),
    "mAP50": float(stats_50[0]) if len(stats_50) > 0 else float('nan'),
    "AP50_fullmetric": float(stats_full[1]) if len(stats_full) > 1 else float('nan'),  # if needed
    "AP75_fullmetric": float(stats_full[2]) if len(stats_full) > 2 else float('nan'),
    "IoU_mean": overall_iou,
    "Dice_mean": overall_dice,
    "n_samples": sum(len(v) for v in iou_scores.values())
}

# DataFrame and save
df = pd.DataFrame(rows).sort_values("class_id")
df_overall = pd.DataFrame([overall_row])
df_out = pd.concat([df, df_overall], ignore_index=True, sort=False)

os.makedirs(os.path.dirname(CSV_OUT) or ".", exist_ok=True)
df_out.to_csv(CSV_OUT, index=False)
print(f"\n📁 Metrics saved to: {CSV_OUT}")

# Print short summary
print("\nPer-class summary (first lines):")
print(df_out.head(30).to_string(index=False))
print("\nOverall mAP (0.5:0.95):", overall_row["mAP"])
print("Overall mAP@50:", overall_row["mAP50"])


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
[11/23 14:19:13 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./out_maskrcnn/model_final.pth ...


Evaluating val images: 100%|██████████| 26/26 [00:10<00:00,  2.50it/s]



--- COCOeval: full mAP (IoU=0.50:0.95) ---
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=0.13s).
Accumulating evaluation results...
DONE (t=0.11s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.297
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.441
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.362
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.406
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.318
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.208
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.382
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.426
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.426
 Average Recall     (AR)